# <font color = teal>  Introduction to testing and evaluating models </font>

Testing phase can be performed either with a single yaml file or with several yaml files located in the same directory.

To make predictions with a trained model, you'll need a csv file to tell a model which parts of the data (i.e., which ECGs) are used as a testing set, and a yaml file which names this csv file for the model. The csv file(s) can be created by following the introductions in the notebook [Introduction to data handling](1_introduction_data_handling.ipynb). Yaml files can be created with the notebooks [Yaml files of database-wise split for training and testing](2_physionet_DBwise_yaml_files.ipynb) and [Yaml files of stratified split for training and testing](2_physionet_stratified_yaml_files.ipynb). And of course, you should have trained a model to test it: To do this, you can find some introduction in the notebook [Introduction to training models](3_introduction_training.ipynb). 

-----------------

<font color ='red'> **NOTE!** </font> *Before you start testing the models, especially when you have made predictions multiple times, check the saving directory that it either contains the predictions you have made in the previous iteration or is empty. If you use different test data with the same yaml file, you might end up having predictions from different csv files and evaluation doesn't work. Mind this especially, if you get an **AssertionError**.*

As with the training phase, yaml files are also used in the prediction and evaluation phase. They have a structure as follows (`predict_smoke.yaml`)

```
# INITIAL SETTINGS
test_file: test_split_1.csv
model: split_1_1.pth

# TESTING SETTINGS
threshold: 0.500000

# DEVICE CONFIGS
device_count: 1

```

where `test_file` refers to the csv file of the test data and `model` refers to the file of the trained model which you want to test. The testing settings include decision threshold. 

<font color ='red'>**NOTE!**</font> The `device_count` attribute should be considered. It refers to the number of GPUs which are used in prediction, similarly as in the training phase.

The script for this phase is `run_model.py`. You should first check the path `csv_root` that it points to the right location in the `data` directory (and its subdirectories). The attribute is set to find the csv file of the testing data. Note that as it's concatenated with the `os.path.join` method, you may only need to change the last path component. `model` will be searched from the `experiments` directory automatically so only the name of the file is nessessary.

### <font color = teal> Saving the results </font>

The predictions are saved as csv files with the following structure

```
#Record ID
164889003, 270492004, 164909002, 426783006, 59118001, 284470004,  164884008,
        1,         1,         0,         0,         0,        0,          0,        
      0.9,       0.6,       0.2,       0.05,      0.2,      0.35,       0.35,  
```

where the first row, `#Record ID`, refers to the file name from which the prediction is made, and the second to the class labels used in SNOMED CT codes, and the third row to the predicted label in binary form (1 - patient is predicted to have the diagnosis above, and 0 - the opposite), and the fourth row to the probability scores for each predicted label. 

The script automatically performs the evaluation automatically after the predictions are made. In the evaluation phase, `metrics.py` is used from `/src/modeling/` to compute the wanted metrics. The evaluation metrics will be saved in the same directory as the predictions as a `pickle` file. ROC curves are also drawn as previously, and they are saved in a `png` format. 

By default, evaluation history will contain micro/macro AUROC scores and micro/macro average precision scores as well as Physionet Challenge 2021 scoring metric (more information about this scoring metric can be found [here under **Scoring**](https://moody-challenge.physionet.org/2021/)). Also, decision threshold, labels and path for the test csv file are saved. These can be modified in the `predict_utils` script in `./src/modeling/` if wanted. There are a `history` dictionary located in the first lines of the `predict` function.

If other metrics are wanted to be computed, you can compute them in the `metrics` scripts in `./src/modeling/`. There can be found the `cal_multilabel_metrics` function, in which the wanted metrics are computed and where you can add new functions. The predicted labels are also one-hot-encoded there, so this format of the predictions is available.

### <font color = teal> Terminal commands </font>

Run a terminal command which consist of the script and the yaml file *or* the directory where all the yaml files are located, so one of the followings

```
python run_model.py predict_smoke.yaml
python run_model.py predict_stratified_smoke
```

---------------------

## <font color = teal> Example: Smoke testing </font>

<font color = red>**NOTE!**</font> <font color = green> **Here, the assumption is that *the data is preprocessed*. If that's not the case, you should use, for example, the original data directory, such as the** `smoke_data` **directory.** The paths for ECGs will be different in the csv files depending on whether preprocessing has been used or not.</font>

### <font color = teal> One yaml file </font>

The yaml file for smoke testing --- `predict_smoke.yaml` --- is available in `/configs/predicting/`. Make sure that the model is trained first and is named as `train_smoke.pth`.

*And before anything, check if there exists a directory named `predict_smoke` in the `experiments` directory. If there are other predictions made and they are not the ones from the files listed below, evaluation won't work correctly. Mind this especially when you get an **AssertionError**.*

The csv file `test_split_1.csv` has the following structure

```
path,age,gender,fs,426783006,426177001,164934002,427084000,164890007,39732003,164889003,59931005,427393009,270492004
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/A0004_preprocessed.mat,45.0,Male,500.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/A0003_preprocessed.mat,81.0,Female,500.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/A0007_preprocessed.mat,74.0,Male,500.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/Q0001_preprocessed.mat,53.0,Male,500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/A0009_preprocessed.mat,81.0,Male,500.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
./data/physionet_preprocessed_smoke/CPSC_CPSC-Extra/A0002_preprocessed.mat,49.0,Female,500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
```

so total of six files are considered as the testing data. All of them are from the CPSC and CPSC-Extra databases.

The path in `run_model.py` should be set as below (note the last path component):

```
csv_root = os.path.join('data', 'split_csvs', 'stratified_smoke')
```

Then you can just run the command to make the predictions with the trained model saved as `train_smoke.pth` as

```
python run_model.py predict_smoke.yaml
```

The predictions can be found in the `predict_smoke` subdirectory of the `experiments` directory. Each prediction is named after the original file name from which the predictions have been made. In smoke testing, they are as follows

```
A0002_preprocessed.csv
A0003_preprocessed.csv
A0004_preprocessed.csv
A0007_preprocessed.csv
A0009_preprocessed.csv
Q0001_preprocessed.csv
```

Each have the structure of the one presented above. 

Wanted metrics (micro and macro average precisions and micro and macro AUROCs) are saved in the file `test_history.pickle` and can be found in the same directory as the predictions are located. The ROC curve is saved as `roc-test.png`.


### <font color = teal> Multiple yaml files in a directory </font>

The idea is similar here: Now you should locate all the yaml files in one directory. The `predict_stratified_smoke` directory in `/configs/predicting/` was created when the yaml files were created with the notebook [Yaml files of stratified split for training and testing](2_physionet_stratified_yaml_files.ipynb). There are four yaml files named as `split_1_1.yaml`, `split_1_2.yaml`, `split_1_3.yaml` and `split_1_4.yaml` in the directory. Each yaml file has the same test file as they've been created using the same stratificated data split, but they do have different models to use for the testing phase. For example, the two first files has the following content:

`split_1_1.yaml`:
```
# INITIAL SETTINGS
test_file: test_split_1.csv
model: split_1_1.pth

# TESTING SETTINGS
threshold: 0.500000

# DEVICE CONFIGS
device_count: 1

```

`split_1_2.yaml`:
```
# INITIAL SETTINGS
test_file: test_split_1.csv
model: split_1_2.pth

# TESTING SETTINGS
threshold: 0.500000

# DEVICE CONFIGS
device_count: 1
```

The path in `run_model.py` should be set as below (note the last path component):

```
csv_root = os.path.join('data', 'split_csvs', 'stratified_smoke')
```

Terminal command for testing is now

```
python run_model.py predict_stratified_smoke
```

The predictions can be found from two subdirectories of the `predict_stratified_smoke` directory in `/experiments/` named as used yaml files. Also the evaluation metrics are saved in both subdirectories in `pickle` format, and the ROC curves are saved as `png` pictures.